# Setup

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Load Vector Data

```
set gmu-buildings gis:load-dataset "data/Campus_data/Mason_bld.shp"

set gmu-walkway gis:load-dataset "data/Campus_data/Mason_walkway_line.shp"

set world-size gis:load-dataset "data/Campus_data/world.shp"
```

In [2]:
gmu_buildings = gpd.read_file("../data/raw/campus/Mason_bld.shp").set_index("Id")
gmu_walkway = gpd.read_file("../data/raw/campus/Mason_walkway_line.shp").set_index("Id")
world_size = gpd.read_file("../data/raw/campus/world.shp").set_index("Id")

gmu_buildings

,NAME,function,geometry
Id,,,
1,GEORGE W. JOHNSON CENTER,1.0,"POLYGON ((11822517.11166 6987684.10842, 118225..."
2,DAVID KING HALL,1.0,"POLYGON ((11822800.55434 6987724.17504, 118227..."
3,SCIENCE AND TECH I,1.0,"POLYGON ((11823106.15955 6987428.08606, 118230..."
4,SCIENCE AND TECH II,1.0,"POLYGON ((11823144.77700 6987485.86687, 118231..."
5,ENTERPRISE HALL,1.0,"POLYGON ((11822946.56417 6987144.44857, 118229..."
...,...,...,...
57,CROSS COTTAGE,NaN,"POLYGON ((11821649.40986 6987068.51409, 118216..."
58,CENTER FOR STUDENT ENGAGEMENT (SUB II),NaN,"POLYGON ((11823355.05141 6987869.56873, 118233..."
59,SANDY CREEK PARKING DECK,NaN,"POLYGON ((11823486.80798 6987357.73456, 118234..."


In [3]:
gmu_walkway

,geometry
Id,
0,"LINESTRING (11822464.14699 6988476.89551, 1182..."
0,"LINESTRING (11822594.10614 6988242.26927, 1182..."
0,"LINESTRING (11822329.18942 6988193.98445, 1182..."
0,"LINESTRING (11822698.07345 6987962.35727, 1182..."
0,"LINESTRING (11822695.07439 6987952.56035, 1182..."
...,...
0,"LINESTRING (11823370.26496 6987341.02044, 1182..."
0,"LINESTRING (11823370.26496 6987341.02044, 1182..."
0,"LINESTRING (11822149.47828 6987062.23034, 1182..."


In [4]:
world_size

,geometry
Id,
0,"POLYGON ((11821007.29556 6989187.46574, 118265..."
